# Projet_P4 : Analysez les ventes de votre entreprise

## Script data cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pylab 
from datetime import datetime
import scipy.stats as stats
from scipy import stats as st
from scipy.stats import pearsonr
%matplotlib inline
# pip install pandas_profiling
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')
# pip install --upgrade pingouin
import pingouin as pg
# pip install researchpy
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols

##### Téléchargement de tous les datasets

In [2]:
url = 'INPUTS_FORMATION/transactions.csv'
data1 = pd.read_csv(url, sep=',',encoding='utf-8')
df1 = data1.copy()

url = 'INPUTS_FORMATION/customers.csv'
data2 = pd.read_csv(url, sep=',',encoding='utf-8')
df2 = data2.copy()

url = 'INPUTS_FORMATION/products.csv'
data3 = pd.read_csv(url, sep=',',encoding='utf-8')
df3 = data3.copy()

##### Pandas Profiling

In [3]:
# prof = ProfileReport(data1) #analyse du csv
# prof.to_file(output_file='rapport_transactions.html')
# prof = ProfileReport(data2) #analyse du csv
# prof.to_file(output_file='rapport_customers.html')
# prof = ProfileReport(data3) #analyse du csv
# prof.to_file(output_file='rapport_products.html')

##### Jointure des 3 datasest

In [4]:
# on aggrége toutes nos données
table = pd.merge(df1, df2, how="left", on="client_id")
table = pd.merge(table, df3, how="left", on="id_prod")
table.head()

,id_prod,date,session_id,client_id,sex,birth,price,categ
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,f,1977,4.99,0.0
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,f,2000,65.75,2.0
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,f,1979,10.71,1.0
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,m,1963,4.20,0.0
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242,f,1980,8.99,0.0


##### Infos générales dataset

In [5]:
# on jette un oeil sur l'aspect général de notre dataset
table.describe()

,birth,price,categ
count,337016.000000,336913.000000,336913.000000
mean,1977.837150,17.204376,0.429900
std,13.531686,17.855658,0.590999
min,1929.000000,-1.000000,0.000000
25%,1971.000000,8.580000,0.000000
50%,1980.000000,13.900000,0.000000
75%,1987.000000,18.990000,1.000000
max,2004.000000,300.000000,2.000000


In [6]:
# on affiche les bornes des achats ainsi que le dataset pour des valeurs d'achats < 0
print('achat min :',table['price'].min(),'\nachat max :',table['price'].max())
invest1 = table.loc[(table['price']==-1)]
invest1      

achat min : -1.0 
achat max : 300.0


,id_prod,date,session_id,client_id,sex,birth,price,categ
1431,T_0,test_2021-03-01 02:30:02.237420,s_0,ct_1,m,2001,-1.0,0.0
2365,T_0,test_2021-03-01 02:30:02.237446,s_0,ct_1,m,2001,-1.0,0.0
2895,T_0,test_2021-03-01 02:30:02.237414,s_0,ct_1,m,2001,-1.0,0.0
5955,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_0,f,2001,-1.0,0.0
7283,T_0,test_2021-03-01 02:30:02.237434,s_0,ct_1,m,2001,-1.0,0.0
...,...,...,...,...,...,...,...,...
332594,T_0,test_2021-03-01 02:30:02.237445,s_0,ct_0,f,2001,-1.0,0.0
332705,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1,m,2001,-1.0,0.0
332730,T_0,test_2021-03-01 02:30:02.237421,s_0,ct_1,m,2001,-1.0,0.0
333442,T_0,test_2021-03-01 02:30:02.237431,s_0,ct_1,m,2001,-1.0,0.0


In [7]:
# on recherche tous les produits dont la désignation n'est pas conforme à l'ensemble (commençant par 'T_')
invest2 = table.loc[(table['id_prod'].str.startswith('T_', na=False))]
print('NB of price negative values :', len(table.loc[(table['price']==-1)]))           
print('NB of id_prod starting with T_ :',len(table.loc[(table['id_prod'].str.startswith('T_', na=False))])) 

# on recherche une potentielle correspondance entre ces 2 'familles' 
invest1['idMatch?'] = np.where((invest1['client_id'] == invest2['client_id'])&(invest1['session_id'] == invest2['session_id']), 'True', 'False')
print('\x1b[6;31;40m','Matching between both families :',invest1['idMatch?'].iat[0],'is unique value','\x1b[0m')   

table_alt1 = table.copy()

# on affiche les valeurs manquantes
print(table_alt1.isnull().sum())

NB of price negative values : 200
NB of id_prod starting with T_ : 200
 Matching between both families : True is unique value 
id_prod         0
date            0
session_id      0
client_id       0
sex             0
birth           0
price         103
categ         103
dtype: int64


## TRAITEMENT 1 (imputation par la médiane)

##### VALEURS MANQUANTES  - TRAITEMENT 1 : imputation par la médiane

In [8]:
# on affiche le(s) produit(s) pour le(s)quel(s) nous n'avons pas de montant d'achat connu
print('\x1b[6;31;40m','Id produit aux données manquantes :',str(table_alt1.loc[table_alt1['price'].isnull(), 'id_prod'].unique().tolist()),'\x1b[0m')           

 Id produit aux données manquantes : ['0_2245'] 


In [9]:
# Imputation par la médiane [price]
median = table_alt1['price'].median()
table_alt1['price'].fillna(round(median,1), inplace=True) 
# Imputation par la valeur connue pour le produit identifié ['0_2245'] => categorie = 0
cat0 = 0.0
table_alt1['categ'].fillna(cat0, inplace=True)

##### VALEURS ABERRANTES  - TRAITEMENT 1 : Suppressions des lignes à données aberrantes

In [10]:
# on supprime les lignes où price < 0 (correspondant au label "test")
shape_init = table.shape
print('\x1b[6;31;40m','SHAPE initiale :',str(shape_init[0]),'\x1b[0m')           
table_alt1.drop(table_alt1.loc[table_alt1['price']== -1 ].index, inplace=True)
shape_fin= table_alt1.shape
Delt = shape_init[0]-shape_fin[0]
print('\x1b[6;31;40m','SHAPE finale   :',str(shape_fin[0]),'\x1b[0m')           
print('\x1b[6;31;40m','Données supprimées : %.2f' %(100*Delt/shape_init[0])+ " % ,",str(Delt),'lignes aberrantes','\x1b[0m')  
# table_alt1.head()

 SHAPE initiale : 337016 
 SHAPE finale   : 336816 
 Données supprimées : 0.06 % , 200 lignes aberrantes 


##### Formatage colonnes date/heure (split)

In [11]:
# on split [date] -> [date] + [heure]
table_alt1[['date', 'heure']] = table_alt1['date'].str.split(' ', n=1, expand=True)

# on convertit & reformate
table_alt1['date']  = pd.to_datetime(table_alt1['date'])    # conversion D en datetime
table_alt1['heure'] = pd.to_timedelta(table_alt1['heure'])  # conversion H en timedelta
table_alt1['heure'] = table_alt1['heure'].dt.floor('s')     # format secondes tronqué

# on conserve seulement les colonnes suivantes
table_alt1 = table_alt1[['id_prod','session_id','client_id','date','heure','sex','birth','categ','price']]

# on crée des colonnes supplémentaires
table_alt1['année'] = table_alt1['date'].dt.year
table_alt1['mois']  = table_alt1['date'].dt.month
table_alt1['age']   = 2022 - table_alt1['birth']
# table_alt1.head()

##### Export de la table

In [12]:
table_alt1.to_csv("OUTPUTS_FORMATION/P4_table_imputations_mediane.csv", index = False)

## TRAITEMENT 2 (imputation par la moyenne)

##### VALEURS ABERRANTES  - TRAITEMENT 2 idem TRAITEMENT 1 (Suppression)

In [13]:
table_alt2 = table.copy()

# on supprime les lignes où price < 0 (correspondant au label "test")
shape_init = table_alt2.shape
print('\x1b[6;31;40m','SHAPE initiale :',str(shape_init[0]),'\x1b[0m')  
shape_init= table_alt2.shape
table_alt2.drop(table_alt2.loc[table_alt2['price']== -1 ].index, inplace=True)
shape_fin= table_alt2.shape
Delt = shape_init[0]-shape_fin[0]
print('\x1b[6;31;40m','SHAPE finale   :',str(shape_fin[0]),'\x1b[0m')           
print('\x1b[6;31;40m','Données supprimées : %.2f' %(100*Delt/shape_init[0])+ " % ,",str(Delt),'lignes aberrantes','\x1b[0m')  

 SHAPE initiale : 337016 
 SHAPE finale   : 336816 
 Données supprimées : 0.06 % , 200 lignes aberrantes 


##### VALEURS MANQUANTES  - TRAITEMENT 2 : imputation par la moyenne

In [14]:
# Imputation par la moyenne [price]
average = table_alt2['price'].mean()
table_alt2['price'].fillna(round(average,1), inplace=True)
# Imputation par la valeur connue pour le produit identifié ['0_2245'] => categorie = 0
cat0 = 0.0
table_alt2['categ'].fillna(cat0, inplace=True)

In [15]:
# IMPUTATION DANS LE CAS Où LA CATEG DU PRODUIT '0_2245' N'ETAIT PAS CONNUE
# # on récupère les proportions du dataset par catégories
# print('\x1b[6;31;40m','Distri categ 0.0: %.0f' %(100*table_alt2['categ'].value_counts(normalize=True)[0]),'%','\x1b[0m')   
# print('\x1b[6;31;40m','Distri categ 1.0: %.0f' %(100*table_alt2['categ'].value_counts(normalize=True)[1]),'%','\x1b[0m')   
# print('\x1b[6;31;40m','Distri categ 2.0: %.0f' %(100*table_alt2['categ'].value_counts(normalize=True)[2]),'%','\x1b[0m')   

# # on travaille sur le df du produit identifié
# df_imputation_unkprod = table_alt2[table_alt2['id_prod']=='0_2245']

# # on impute les valeurs de catégories manquantes suivant cette répartition (nb rows => c0 : 64 , c1 : 34 , c2 : 5)
# distribution = 64*[0.0]+34*[1.0]+5*[2.0]
# df_imputation_unkprod.loc[:,'categ'] = distribution
# df_imputation_unkprod.head()

# # on revient au dataset global, on supprime et remplace avec les nouvelles imputations pour le produit identifié
# table_alt2.drop(table_alt2.loc[table_alt2['id_prod']== '0_2245' ].index, inplace=True)

# # on concatène les 2 datasets
# frames = [table_alt2, df_imputation_unkprod]
# table_alt2 = pd.concat(frames)

##### Formatage colonnes date/heure (split)

In [16]:
# on renomme l'unique valeur [date] commençant par 'test_'
table_alt2['date'] = table_alt2['date'].str.replace('test_2021-03-01' ,'2021-03-01')

# on split [date] -> [date] + [heure]
table_alt2[['date', 'heure']] = table_alt2['date'].str.split(' ', n=1, expand=True)

# on convertit & reformate
table_alt2['date']  = pd.to_datetime(table_alt2['date'])    # conversion D en datetime
table_alt2['heure'] = pd.to_timedelta(table_alt2['heure'])  # conversion H en timedelta
table_alt2['heure'] = table_alt2['heure'].dt.floor('s')     # format secondes tronqué

# on conserve seulement les colonnes suivantes
table_alt2 = table_alt2[['id_prod','session_id','client_id','date','heure','sex','birth','categ','price']]

# on crée des colonnées supplémentaires
table_alt2['année'] = table_alt2['date'].dt.year
table_alt2['mois']  = table_alt2['date'].dt.month
table_alt2['age']   = 2022 - table_alt2['birth']

##### Export de la table

In [17]:
table_alt2.to_csv("OUTPUTS_FORMATION/P4_table_imputations_moyenne.csv", index = False)